<a href="https://colab.research.google.com/github/Byeon-MJ/ML_Practice_Repo/blob/main/PCA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 주성분 분석(Principal Component Analysis)

## 차원과 차원 축소
* 차원 축소(Dimensionality Reduction) : 데이터를 나타내는 일부 특성을 선택하여 데이터 크기를 줄이고 지도학습 모델의 성능을 향상시키는 방법
* 특성이 많으면 선형 모델의 성능은 높아지지만 훈련 데이터에 쉽게 과대적합 됨

## 주성분 분석 소개
* PCA : 데이터에 있는 분산이 큰 방향을 찾는 것
* 분산이 큰 방향이란 데이터를 잘 표현하는 어떤 벡터
* 일반적으로 주성분은 원본 특성의 개수만큼 찾을 수 있다.

## PCA 클래스

In [ ]:
!wget https://bit.ly/fruits_300_data -O fruits_300.npy
import numpy as np
fruits = np.load('fruits_300.npy')
fruits_2d = fruits.reshape(-1, 100*100)

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=50)
pca.fit(fruits_2d)

In [ ]:
print(pca.components_.shape)

In [ ]:
# 이미지 시각화 함수 정의
import matplotlib.pyplot as plt
def draw_fruits(arr, ratio=1):
    n = len(arr)    # n은 샘플 개수
    # 한 줄에 10개씩 이미지 그리기
    rows = int(np.ceil(n/10))
    cols = n if rows < 2 else 10
    fig, axs = plt.subplots(rows, cols, figsize=(cols*ratio, rows*ratio), squeeze=False)

    for i in range(rows):
        for j in range(cols):
            if i*10 + j < n:
                axs[i, j].imshow(arr[i*10 + j], cmap='gray_r')
            axs[i, j].axis('off')
    plt.show()
     

In [ ]:
draw_fruits(pca.components_.reshape(-1, 100, 100))

### 원본 데이터 차원 줄여보기

In [ ]:
print(fruits_2d.shape)

In [ ]:
fruits_pca = pca.transform(fruits_2d)
print(fruits_pca.shape)

## 원본 데이터 재구성
* 데이터의 특성을 줄렸기 때문에 어느정도 손실이 발생한다.
* 최대한 분산이 큰 방향으로 데이터를 투영했기 때문에 원본 데이터를 상당 부분을 재구성할 수 있다.
* PCA 클래스에서는 inverse_transform() 메서드 제공

In [ ]:
fruits_inverse = pca.inverse_transform(fruits_pca)
print(fruits_inverse.shape)

In [ ]:
fruits_reconstruct = fruits_inverse.reshape(-1, 100, 100)
for start in [0, 100, 200]:
    draw_fruits(fruits_reconstruct[start:start+100])
    print('\n')

## 설명된 분산
* explained_variance_ratio_ : 각 주성분의 설명된 분산 비율이 기록

In [ ]:
print(np.sum(pca.explained_variance_ratio_))

In [ ]:
plt.plot(pca.explained_variance_ratio_)
plt.show()

# 다른 알고리즘과 함께 사용하기
* 로지스틱 회귀 모델과 함께 적용

In [ ]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()

In [ ]:
# target 만들기
target = np.array([0]*100 + [1]*100 + [2]*100)

## 원본 데이터로 로지스틱 회귀모델 성능 검증

In [ ]:
from sklearn.model_selection import cross_validate
scores = cross_validate(lr, fruits_2d, target)
print(np.mean(scores['test_score']))
print(np.mean(scores['fit_time']))

## PCA로 차원 축소된 데이터의 성능 검증

In [ ]:
scores = cross_validate(lr, fruits_pca, target)
print(np.mean(scores['test_score']))
print(np.mean(scores['fit_time']))

## 특정 분산비율로 PCA 하기

In [ ]:
# 분산 비율로 pca 하기
pca = PCA(n_components=0.5)
pca.fit(fruits_2d)

In [ ]:
print(pca.n_components_)

In [ ]:
fruits_pca = pca.transform(fruits_2d)
print(fruits_pca.shape)

In [ ]:
# 2개의 주성분 교차검증 성능 확인
scores = cross_validate(lr, fruits_pca, target)
print(np.mean(scores['test_score']))
print(np.mean(scores['fit_time']))

## K-Means 알고리즘으로 클러스터 찾기

In [ ]:
from sklearn.cluster import KMeans
km = KMeans(n_clusters=3, random_state=42)
km.fit(fruits_pca)
print(np.unique(km.labels_, return_counts=True))

In [ ]:
for label in range(3):
    draw_fruits(fruits[km.labels_ == label])
    print('\n')

In [ ]:
for label in range(3):
    data = fruits_pca[km.labels_ == label]
    plt.scatter(data[:, 0], data[:, 1])
plt.legend(['pineapple', 'banana', 'apple'])
plt.show()